In [5]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing

In [6]:
raw_data= np.loadtxt("Audiobooks_data.csv", delimiter=",")
unscaled_inputs_all=raw_data[:,1:-1]
targets_all=raw_data[:,-1]

In [7]:
num_one_targets=int(np.sum(targets_all))
zero_targets_counter=0
indices_to_remove=[]
for i in range(targets_all.shape[0]):
    if targets_all[i]==0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
unscaled_inputs_equal_priors=np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors=np.delete(targets_all, indices_to_remove, axis=0)

In [8]:
scaled_inputs=preprocessing.scale(unscaled_inputs_equal_priors)

In [9]:
shuffled_indices=np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs=scaled_inputs[shuffled_indices]
shuffled_targets=targets_equal_priors[shuffled_indices]

In [10]:
samples_count=shuffled_inputs.shape[0]
train_samples_count= int(0.8*samples_count)
validation_samples_count=int(0.1*samples_count)
test_samples_count=samples_count-(train_samples_count + validation_samples_count)

train_inputs=shuffled_inputs[:train_samples_count]
train_targets=shuffled_targets[:train_samples_count]

validation_inputs=shuffled_inputs[train_samples_count:train_samples_count+ validation_samples_count]
validation_targets=shuffled_targets[train_samples_count:train_samples_count+ validation_samples_count]

test_inputs=shuffled_inputs[train_samples_count+ validation_samples_count:]
test_targets=shuffled_targets[train_samples_count+ validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets)/train_samples_count )
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets)/validation_samples_count )
print(np.sum(test_targets), test_samples_count, np.sum(test_targets)/test_samples_count )

1785.0 3579 0.49874266554903607
235.0 447 0.5257270693512305
217.0 448 0.484375


In [11]:
np.savez("Audiobook_data_train", inputs=train_inputs, targets=train_targets)
np.savez("Audiobook_data_validation", inputs=validation_inputs, targets=validation_targets)
np.savez("Audiobook_data_test", inputs=test_inputs, targets=test_targets)

In [12]:
npz= np.load("Audiobook_data_train.npz")
train_inputs=npz["inputs"].astype(np.float64)
train_targets=npz["targets"].astype(np.int64)

npz=np.load("Audiobook_data_validation.npz")
validation_inputs=npz["inputs"].astype(np.float64)
validation_targets=npz["targets"].astype(np.int64)

npz=np.load("Audiobook_data_test.npz")
test_inputs=npz["inputs"].astype(np.float64)
test_targets=npz["targets"].astype(np.int64)

In [13]:
input_size=10
output_size=2
hidden_layer_size=50
model= tf.keras.Sequential([
                    tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
                    tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
                    tf.keras.layers.Dense(output_size, activation="softmax")
])
model.compile(optimizer='adam', loss= 'sparse_categorical_crossentropy', metrics=['accuracy'])
batch_size=100
max_epochs=100
early_stopping=tf.keras.callbacks.EarlyStopping(patience=2)
model.fit(train_inputs, train_targets, batch_size=batch_size,callbacks=[early_stopping], epochs=max_epochs, validation_data=(validation_inputs,validation_targets), verbose=2)

Epoch 1/100
36/36 - 1s - loss: 0.6234 - accuracy: 0.6695 - val_loss: 0.5735 - val_accuracy: 0.7025 - 705ms/epoch - 20ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4988 - accuracy: 0.7508 - val_loss: 0.4824 - val_accuracy: 0.7248 - 94ms/epoch - 3ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4263 - accuracy: 0.7762 - val_loss: 0.4257 - val_accuracy: 0.7919 - 86ms/epoch - 2ms/step
Epoch 4/100
36/36 - 0s - loss: 0.3915 - accuracy: 0.7837 - val_loss: 0.4011 - val_accuracy: 0.7718 - 90ms/epoch - 3ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3743 - accuracy: 0.7969 - val_loss: 0.3908 - val_accuracy: 0.7830 - 83ms/epoch - 2ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3624 - accuracy: 0.8061 - val_loss: 0.3729 - val_accuracy: 0.7852 - 83ms/epoch - 2ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3543 - accuracy: 0.8092 - val_loss: 0.3760 - val_accuracy: 0.7785 - 89ms/epoch - 2ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3463 - accuracy: 0.8092 - val_loss: 0.3636 - val_accuracy: 0.7852 - 99ms/epoch - 3ms/step
Epoch 9/100
36

In [14]:
test_loss, test_accuracy=model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.3473 - accuracy: 0.8326
